In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 6000
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 6000
N_FEATURES = 8

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 160, 16

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 40.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [14]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [18]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [19]:
# import os
path='/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/'

In [20]:
for i in range(1,18):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
       
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 20, 300
    n_depth=8
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
   # chen architecture
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=32, kernel_size=2, padding='same', kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04)), \
                               input_shape=(n_steps,n_length,n_depth)))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Conv1D(filters=16, kernel_size=2,padding="same",kernel_initializer="he_normal",strides=1,kernel_regularizer=l1(1e-04))))
    model.add(TimeDistributed(BatchNormalization(epsilon=1e-06, momentum=0.95, weights=None)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4,strides=2)))
    model.add(TimeDistributed(Activation('tanh')))
    model.add(TimeDistributed(Dropout(0.2093)))
    model.add(TimeDistributed(Activation('relu')))
    model.add(TimeDistributed(Flatten()))
    # model.add(Flatten())
    model.add(LSTM(64,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(32,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='tanh'))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.9, weights=None))
    model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    print('test_mean for %d subjects:',i)
    test_mean=statistics.mean(test_acc)
    print(test_mean)

/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 312000

/media/naveen/nav/mat_codes/BioPatRec/olsson_prep_WOR_BP_D2/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 9
Number of rows in the dataframe: 156000

x_train shape:  (51, 6000, 8)
51 training samples
y_train shape:  (51,)
num_time_periods 6000
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0]
x_train shape: (6000, 8)
input_shape: (6000, 8)
y_train shape (51,)
New y_train shape:  (51, 26)
x_test shape:  (25, 6000, 8)
25 testing samples
y_test shape:  (25,)
x_train shape:  (51, 20, 300, 8)
x_test shape:  (25, 20, 300, 8)
n_outputs 26
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param # 

4/4 [==============================] - 0s 30ms/step - loss: 1.6503 - accuracy: 0.4902 - val_loss: 1.3235 - val_accuracy: 0.5600 - lr: 0.0010
Epoch 15/160
1/4 [======>.......................] - ETA: 0s - loss: 1.5052 - accuracy: 0.5625
Epoch 00015: val_accuracy improved from 0.56000 to 0.68000, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
4/4 [==============================] - 0s 28ms/step - loss: 1.5126 - accuracy: 0.5294 - val_loss: 1.1760 - val_accuracy: 0.6800 - lr: 0.0010
Epoch 16/160
1/4 [======>.......................] - ETA: 0s - loss: 1.6700 - accuracy: 0.2500
Epoch 00016: val_accuracy improved from 0.68000 to 0.72000, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
4/4 [==============================] - 0s 27ms/step - loss: 1.5992 - accuracy: 0.4118 - val_loss: 1.1006 - val_accuracy: 0.7200 - lr: 0.0010
Epoch 17/160
1/4 [======>.......................] - ETA: 0s - loss: 1.6191 - 

1/4 [======>.......................] - ETA: 0s - loss: 0.6736 - accuracy: 0.8750
Epoch 00041: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 23ms/step - loss: 0.7152 - accuracy: 0.8431 - val_loss: 0.6713 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 42/160
1/4 [======>.......................] - ETA: 0s - loss: 0.9481 - accuracy: 0.6250
Epoch 00042: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 20ms/step - loss: 0.8208 - accuracy: 0.7451 - val_loss: 0.6650 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 43/160
1/4 [======>.......................] - ETA: 0s - loss: 0.8896 - accuracy: 0.8750
Epoch 00043: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 19ms/step - loss: 0.8198 - accuracy: 0.8235 - val_loss: 0.6490 - val_accuracy: 0.8000 - lr: 1.0000e-04
Epoch 44/160
1/4 [======>.......................] - ETA: 0s - loss: 0.6873 - accuracy: 0.8750
Epoch 00044: val_accuracy did not imp

Epoch 69/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4673 - accuracy: 0.8750
Epoch 00069: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 28ms/step - loss: 0.5456 - accuracy: 0.9020 - val_loss: 0.5599 - val_accuracy: 0.8400 - lr: 1.0000e-04
Epoch 70/160
4/4 [==============================] - ETA: 0s - loss: 0.6525 - accuracy: 0.8824
Epoch 00070: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 33ms/step - loss: 0.6525 - accuracy: 0.8824 - val_loss: 0.5513 - val_accuracy: 0.8400 - lr: 1.0000e-04
Epoch 71/160
4/4 [==============================] - ETA: 0s - loss: 0.5886 - accuracy: 0.8627
Epoch 00071: val_accuracy did not improve from 0.84000
4/4 [==============================] - 0s 37ms/step - loss: 0.5886 - accuracy: 0.8627 - val_loss: 0.5438 - val_accuracy: 0.8400 - lr: 1.0000e-04
Epoch 72/160
1/4 [======>.......................] - ETA: 0s - loss: 0.5567 - accuracy: 0.8750
Epoch 00072: val_accurac

Epoch 97/160
1/4 [======>.......................] - ETA: 0s - loss: 0.5543 - accuracy: 0.9375
Epoch 00097: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 29ms/step - loss: 0.5814 - accuracy: 0.8431 - val_loss: 0.5297 - val_accuracy: 0.8800 - lr: 1.0000e-05
Epoch 98/160
4/4 [==============================] - ETA: 0s - loss: 0.5584 - accuracy: 0.8627
Epoch 00098: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 31ms/step - loss: 0.5584 - accuracy: 0.8627 - val_loss: 0.5313 - val_accuracy: 0.8800 - lr: 1.0000e-05
Epoch 99/160
4/4 [==============================] - ETA: 0s - loss: 0.6263 - accuracy: 0.8431
Epoch 00099: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 32ms/step - loss: 0.6263 - accuracy: 0.8431 - val_loss: 0.5464 - val_accuracy: 0.7600 - lr: 1.0000e-05
Epoch 100/160
1/4 [======>.......................] - ETA: 0s - loss: 0.6347 - accuracy: 0.8125
Epoch 00100: val_accura

...

Epoch 138/160
1/4 [======>.......................] - ETA: 0s - loss: 0.7561 - accuracy: 0.7500
Epoch 00138: val_accuracy did not improve from 0.68000
4/4 [==============================] - 0s 21ms/step - loss: 0.5857 - accuracy: 0.8627 - val_loss: 1.0925 - val_accuracy: 0.5600 - lr: 1.0000e-06
Epoch 139/160
1/4 [======>.......................] - ETA: 0s - loss: 0.5930 - accuracy: 0.9375
Epoch 00139: val_accuracy did not improve from 0.68000
4/4 [==============================] - 0s 20ms/step - loss: 0.5260 - accuracy: 0.9608 - val_loss: 1.0790 - val_accuracy: 0.6000 - lr: 1.0000e-06
Epoch 140/160
1/4 [======>.......................] - ETA: 0s - loss: 0.5011 - accuracy: 0.8750
Epoch 00140: val_accuracy did not improve from 0.68000
4/4 [==============================] - 0s 20ms/step - loss: 0.6354 - accuracy: 0.8431 - val_loss: 1.1150 - val_accuracy: 0.5600 - lr: 1.0000e-06
Epoch 141/160
1/4 [======>.......................] - ETA: 0s - loss: 0.6594 - accuracy: 0.8125
Epoch 00141: val_acc

1/4 [======>.......................] - ETA: 0s - loss: 4.5211 - accuracy: 0.0000e+00
Epoch 00001: val_accuracy improved from -inf to 0.16000, saving model to /media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5
4/4 [==============================] - 1s 159ms/step - loss: 4.1163 - accuracy: 0.0196 - val_loss: 3.1943 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 2/160
1/4 [======>.......................] - ETA: 0s - loss: 3.9159 - accuracy: 0.0625
Epoch 00002: val_accuracy did not improve from 0.16000
4/4 [==============================] - 0s 23ms/step - loss: 3.4515 - accuracy: 0.0980 - val_loss: 3.1317 - val_accuracy: 0.1600 - lr: 0.0010
Epoch 3/160
1/4 [======>.......................] - ETA: 0s - loss: 3.6897 - accuracy: 0.0625
Epoch 00003: val_accuracy did not improve from 0.16000
4/4 [==============================] - 0s 21ms/step - loss: 3.1799 - accuracy: 0.1765 - val_loss: 2.9198 - val_accuracy: 0.1200 - lr: 0.0010
Epoch 4/160
1/4 [======>..................

Epoch 53/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4656 - accuracy: 0.8750
Epoch 00053: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 31ms/step - loss: 0.5634 - accuracy: 0.8627 - val_loss: 0.5709 - val_accuracy: 0.8800 - lr: 1.0000e-04
Epoch 54/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4596 - accuracy: 0.9375
Epoch 00054: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 25ms/step - loss: 0.5090 - accuracy: 0.9216 - val_loss: 0.5746 - val_accuracy: 0.8800 - lr: 1.0000e-04
Epoch 55/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4249 - accuracy: 0.8750
Epoch 00055: val_accuracy did not improve from 0.88000
4/4 [==============================] - 0s 27ms/step - loss: 0.5355 - accuracy: 0.8627 - val_loss: 0.5668 - val_accuracy: 0.8800 - lr: 1.0000e-04
Epoch 56/160
4/4 [==============================] - ETA: 0s - loss: 0.4855 - accuracy: 0.9412
Epoch 00056: val_accurac

Epoch 108/160
1/4 [======>.......................] - ETA: 0s - loss: 0.3281 - accuracy: 0.8750
Epoch 00108: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 21ms/step - loss: 0.4188 - accuracy: 0.9020 - val_loss: 0.5059 - val_accuracy: 0.9200 - lr: 1.0000e-05
Epoch 109/160
1/4 [======>.......................] - ETA: 0s - loss: 0.6044 - accuracy: 0.8750
Epoch 00109: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 22ms/step - loss: 0.7137 - accuracy: 0.7843 - val_loss: 0.4931 - val_accuracy: 0.9200 - lr: 1.0000e-05
Epoch 110/160
1/4 [======>.......................] - ETA: 0s - loss: 0.3556 - accuracy: 0.9375
Epoch 00110: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 20ms/step - loss: 0.4778 - accuracy: 0.9216 - val_loss: 0.4954 - val_accuracy: 0.9200 - lr: 1.0000e-05
Epoch 111/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4193 - accuracy: 0.9375
Epoch 00111: val_acc

Epoch 136/160
1/4 [======>.......................] - ETA: 0s - loss: 0.2797 - accuracy: 1.0000
Epoch 00136: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 21ms/step - loss: 0.5067 - accuracy: 0.9020 - val_loss: 0.5212 - val_accuracy: 0.9200 - lr: 1.0000e-06
Epoch 137/160
1/4 [======>.......................] - ETA: 0s - loss: 0.3508 - accuracy: 1.0000
Epoch 00137: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 21ms/step - loss: 0.3584 - accuracy: 0.9608 - val_loss: 0.5353 - val_accuracy: 0.9200 - lr: 1.0000e-06
Epoch 138/160
1/4 [======>.......................] - ETA: 0s - loss: 0.3621 - accuracy: 0.8750
Epoch 00138: val_accuracy did not improve from 0.96000
4/4 [==============================] - 0s 22ms/step - loss: 0.3866 - accuracy: 0.9216 - val_loss: 0.5300 - val_accuracy: 0.9200 - lr: 1.0000e-06
Epoch 139/160
1/4 [======>.......................] - ETA: 0s - loss: 0.4595 - accuracy: 0.8750
Epoch 00139: val_acc

In [21]:
# statistics.mean(test_acc)
test_std=statistics.stdev(test_acc)
print(test_std)

0.07970533868686391
